In [1]:
!nvidia-smi

Sat Jul 11 00:33:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('./gdrive', force_remount=True)
%cd './gdrive/My Drive/cubricks'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive
/content/gdrive/My Drive/cubricks


In [3]:
!pip install -q tensorflow-gpu

     |████████████████████████████████| 516.2MB 30kB/s 


In [4]:
%tensorflow_version 2.x

import os
import numpy as np
import pandas as pd
import tensorflow as tf

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
### Load csv file ###
df = pd.read_csv('InvoicedDocuments_v3.csv', na_values=['N/I'])

### Trim string/Remove duplicates ###
df.columns = df.columns.str.replace(' ', '')
df.drop_duplicates('Document', keep=False, inplace=True)

### String to datetime format ###
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'], errors='coerce')
df['DueDate'] = pd.to_datetime(df['DueDate'], errors='coerce')
df['ClearingDate'] = pd.to_datetime(df['ClearingDate'], errors='coerce')
df['LastCreditReview'] = pd.to_datetime(df['LastCreditReview'], errors='coerce')

### Drop/Fill null values ###
df.dropna(subset=['DocumentDate', 'Document', 'DueDate', 'ClearingDate'], inplace=True)
df[df.columns.drop('LastCreditReview')].fillna(0, inplace=True)

### Float to integer ###
df['Company'] = pd.to_numeric(df['Company'], downcast='integer')
df['Document'] = pd.to_numeric(df['Document'], downcast='integer')
df['CustomerCode'] = pd.to_numeric(df['CustomerCode'], downcast='integer')

### String to number ###
string_number = lambda x: int(''.join(format(ord(w), '') for w in str(x)))
df['Country'] = df['Country'].apply(string_number)
df['Region'] = df['Region'].apply(string_number)
df['Terms'] = df['Terms'].apply(string_number)

### Amount treatment/filter ###
df['Amount'] = df['Amount'].apply(lambda x: int(str(x).replace(',', '')))
df = df[df['Amount'] > 20]

### Summary ###
df.info()
df.head(100)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 269608 entries, 1 to 273941
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Company           269608 non-null  int16         
 1   CustomerCode      269608 non-null  int32         
 2   Country           269608 non-null  int64         
 3   Region            269608 non-null  int64         
 4   LastCreditReview  232054 non-null  datetime64[ns]
 5   DocumentDate      269608 non-null  datetime64[ns]
 6   DueDate           269608 non-null  datetime64[ns]
 7   ClearingDate      0 non-null       float64       
 8   Terms             269608 non-null  int64         
 9   Document          269608 non-null  int32         
 10  Amount            269608 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int16(1), int32(2), int64(4)
memory usage: 21.1 MB


,Company,CustomerCode,Country,Region,LastCreditReview,DocumentDate,DueDate,ClearingDate,Terms,Document,Amount
1,1,72575,6569,6569,NaT,2020-03-02,2020-04-16,NaN,74665253,931595079,17154
2,1,72575,6569,6569,NaT,2020-03-02,2020-04-16,NaN,74665253,931595078,17154
3,1,72575,6569,6569,NaT,2020-03-02,2020-04-16,NaN,74665253,931595080,17154
4,1,72575,6569,6569,NaT,2020-03-02,2020-04-16,NaN,74665253,931585016,17154
5,1,72575,6569,6569,NaT,2020-03-03,2020-04-17,NaN,74665253,931602597,17154
...,...,...,...,...,...,...,...,...,...,...,...
104,1,4750138,6569,6569,NaT,2020-06-19,2020-06-19,NaN,65805148,932283905,7259
105,1,4750138,6569,6569,NaT,2020-06-22,2020-06-22,NaN,65805148,932289778,7259
106,1,4750138,6569,6569,NaT,2020-06-15,2020-06-15,NaN,65805148,932240974,6650
107,1,4750138,6569,6569,NaT,2020-06-15,2020-06-15,NaN,65805148,932240973,6650


In [6]:
# bins = [1, 1000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 200000, np.inf]
# labels = [f'{bins[i]}-{bins[i+1]-1}' for i in range(len(bins[:-1]))]

# df['AmountRange'] = pd.cut(df['Amount'], bins=bins, labels=labels, right=False, include_lowest=True)

# plt.figure(figsize=(30, 5))
# sns.countplot(df['AmountRange'])

In [ ]:
#########################
### Generate DaysTo__ ###
#########################
df['DaysToLastCreditReview'] = df['DocumentDate'] - df['LastCreditReview']
df['DaysToLastCreditReview'] = df['DaysToLastCreditReview'].fillna(pd.Timedelta(seconds=-1)).astype('timedelta64[D]').astype(int)
df = df[df['DaysToLastCreditReview'] >= -1]

df['DaysToDue'] = (df['DueDate'] - df['DocumentDate']).astype('timedelta64[D]').astype(int)
df = df[df['DaysToDue'] > 0]

df['DaysToClearingDate'] = (df['ClearingDate'] - df['DocumentDate']).astype('timedelta64[D]').astype(int)
df = df[df['DaysToClearingDate'] > 0]

df.head(10)

In [ ]:
#####################
### Define IsLate ###
#####################
df['IsLate'] = (df['DueDate'] - df['ClearingDate']).astype('timedelta64[D]').astype(int)
df['IsLate'] = df['IsLate'].apply(lambda x: 0 if x > 0 else 1)

sns.countplot(df['IsLate'])
df.head(10)

In [ ]:
# df = df[df['IsLate'] == 0]
# df.head(10)

In [ ]:
###############################################
### Define ClearingDateRange and Categorize ###
###############################################
bins = [1, 8, 15, 22, 29, np.inf]
labels = [f'{bins[i]}-{bins[i+1]-1}' for i in range(len(bins[:-1]))]

print(f'Labels: {labels}')

df['ClearingDateRange'] = pd.cut(df['DaysToClearingDate'], bins=bins, labels=labels, right=False, include_lowest=True)
df[['ClearingDateRangeCT']] = df[['ClearingDateRange']].apply(lambda x: pd.Categorical(x, ordered=True).codes)

df['DaysToDueRange'] = pd.cut(df['DaysToDue'], bins=bins, labels=labels, right=False, include_lowest=True)
df[['DaysToDueRangeCT']] = df[['DaysToDueRange']].apply(lambda x: pd.Categorical(x, ordered=True).codes)

sns.countplot(df['ClearingDateRange'])
df.head(10)

In [ ]:
################################################
### Extract Document/Due month, day, weekday ###
################################################
df['DocumentMonth'] = df['DocumentDate'].dt.month
df['DocumentDay'] = df['DocumentDate'].dt.day
df['DocumentWeekDay'] = df['DocumentDate'].dt.weekday


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30,4))
sns.countplot(df['DocumentMonth'], ax=ax[0])
sns.countplot(df['DocumentDay'], ax=ax[1])
sns.countplot(df['DocumentWeekDay'], ax=ax[2])

df.head(10)

In [ ]:
y_column = np.array(['ClearingDateRangeCT'])
features = np.array([
                    #  'ClearingDateRangeCT',
                    #  'DaysToClearingDate',
                    #  'Document',
                    #  'DaysToDueRangeCT',
                     'Company',
                     'CustomerCode',
                    #  'Country',
                     'Region',
                     'Terms',
                     'Amount',
                     'DaysToLastCreditReview',
                     'DaysToDue',
                     'IsLate',
                     'DocumentMonth',
                     'DocumentDay',
                     'DocumentWeekDay',
                     ])


plt.figure(figsize=(10, 10))
sns.heatmap(df[features].corr(), square=True, annot=True, fmt='.1g', vmin=-1, vmax=1, center=0, cmap='Pastel1')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[features].values,
                                                    df[y_column].values,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=df[y_column].values)

In [ ]:
# dumb predict
print(f'Total items: {len(y_test)}')

for i in range(len(labels)):
    dumb_data = np.ones(y_test.shape, dtype=np.int) * i
    print(f'Always predicting the range "{i}", accuracy is {accuracy_score(y_test, dumb_data) * 100:.2f}%')

In [ ]:
# print(f'Always predicting the range of "DueRange", accuracy is {accuracy_score(y_test, x_test[:,0]) * 100:.2f}%')

# cm_report = classification_report(y_test, x_test[:,0], target_names=labels)
# print(cm_report)

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42, n_jobs=-1)
random_forest.fit(x_train, np.squeeze(y_train))

In [ ]:
predict = random_forest.predict(x_test)

importances = random_forest.feature_importances_
indices = np.argsort(importances)[::-1]

print(f'Total items: {len(y_test)}')
print(f'Accuracy: {accuracy_score(y_test, predict) * 100:.2f}%\n')
print(f'Feature ranking:')

for f in range(x_train.shape[1]):
    print(f'{importances[indices[f]]}\t{features[indices[f]]}')

plt.barh(range(x_train.shape[1]), importances[indices])
plt.yticks(range(x_train.shape[1]), features[indices])
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
cm_report = classification_report(y_test, predict, target_names=labels)
print(cm_report)

In [ ]:
cm = confusion_matrix(y_test, predict)

plt.figure(figsize=(10, 10))
sns.heatmap(cm, square=True, annot=True, fmt='d', cmap='Blues')

In [ ]:
def binary_encoding(df, columns, left_padding=0):
  for item in columns:
    col = np.array([str('{0:b}'.format(x)) for x in df[item].values])
    maxlength = len(max(col, key=len)) + left_padding

    header = np.array([f'{item}{i}' for i in range(maxlength)])
    newcol = np.zeros((col.shape[0], maxlength), dtype=np.int8)

    for i in range(len(col)):
      a = np.array(list(col[i]), dtype=np.int8)
      newcol[i][maxlength - len(a):] = a

    df2 = pd.DataFrame(newcol, columns=header)

    df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df2], axis=1)

  df.drop(columns=columns, inplace=True)
  return df


df_network = df[features].copy()

category = ['Company',
            'CustomerCode',
            'Region',
            'Terms',
            'DocumentMonth',
            'DocumentDay',
            'DocumentWeekDay']

df_network = binary_encoding(df_network, category)

df_network.head(10)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_network.values,
                                                    df[y_column].values,
                                                    test_size=0.1, 
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=df[y_column].values)

y_train_categorical = tf.keras.utils.to_categorical(y_train)
y_test_categorical = tf.keras.utils.to_categorical(y_test)

In [ ]:
def create_model():
    model = tf.keras.models.Sequential(name='cubricks')

    model.add(tf.keras.layers.Input(shape=x_train.shape[1]))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(rate=0.1))

    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(rate=0.1))

    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(len(labels), activation='softmax'))
    return model


model = create_model()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-8),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1, reduction='none'),
              metrics=['accuracy'])

model.summary()

In [ ]:
logdir = os.path.join('.', 'output')
os.makedirs(logdir, exist_ok=True)

training_log = os.path.join(logdir, 'training.txt')
model_checkpoint = os.path.join(logdir, 'model.hdf5')

if os.path.isfile(model_checkpoint):
    model.load_weights(model_checkpoint)

callbacks = [
    tf.keras.callbacks.TensorBoard(logdir, profile_batch=0),
    tf.keras.callbacks.CSVLogger(training_log, separator=',', append=True),
    tf.keras.callbacks.ModelCheckpoint(model_checkpoint, monitor='val_accuracy', save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', min_delta=1e-8, factor=0.2, patience=100, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-8, patience=200, restore_best_weights=True, verbose=1),
]

In [ ]:
# model.fit(x_train,
#           y_train_categorical,
#           validation_data=(x_test, y_test_categorical),
#           callbacks=callbacks,
#           batch_size=1024,
#           epochs=10000,
#           verbose=1)

In [ ]:
predict = np.argmax(model.predict(x_test), axis=1)

print(f'Total: {len(y_test)}')
print(f'Accuracy: {accuracy_score(y_test, predict) * 100:.2f}%')

In [ ]:
cm_report = classification_report(y_test, predict, target_names=labels)
print(cm_report)

In [ ]:
cm = confusion_matrix(y_test, predict)

plt.figure(figsize=(10, 10))
sns.heatmap(cm, square=True, annot=True, fmt='d', cmap='Blues')